In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [97]:
% mv /Users/codetesting/Downloads/benchmark.zip benchmark.zip

mv: /Users/codetesting/Downloads/benchmark.zip: No such file or directory


In [98]:
summary = pd.read_csv("benchmark_data/index.csv")

In [99]:
hedder = pd.read_csv("benchmark_data/header.csv")

In [100]:
summary.columns = hedder.columns

In [101]:
summary.tail(5)

,lr,reward_lr,epochs,reward_epochs,k,ch_h,agent,num_iters,max_delay,hyperbolic_constant,algorithm,reward_prob,imsize,model,seeds,batchsize,use_gpu,final_accuracy,performance
366,0.025,0.1,30,50,10,150,optimal,50,7,0.9,boltzmann_planner,0.05,8,VIN,1-2-3-4-5-6-7-8,20,True,92.53281252458692,0.43958470492818447
367,0.025,0.1,30,50,10,150,optimal,50,7,0.9,boltzmann_planner,0.05,8,VI,1-2-3-5-8-13-21-34,20,True,None,None
368,0.025,0.1,30,50,10,150,optimal,50,7,0.9,boltzmann_planner,0.05,8,VI,2-3-5-8-13-21-34-55,20,True,None,None
369,0.025,0.1,30,50,10,150,optimal,50,7,0.9,boltzmann_planner,0.05,8,VI,3-5-8-13-21-34-55-89,20,True,None,None
370,0.025,0.1,30,50,10,150,optimal,50,7,0.9,boltzmann_planner,0.05,8,VI,1-2-3-4-5-6-7-8,20,True,None,None


In [102]:
completed_frame = summary.loc[summary[' performance'] != ' None']
completed_frame = completed_frame.copy()

## Number of successful calls

Algorithms:

In [103]:
completed_frame[' algorithm'].value_counts()

 vi_inference         56
 no_rewards           48
 given_rewards        47
 boltzmann_planner    40
Name:  algorithm, dtype: int64

Models:

In [104]:
completed_frame[' model'].value_counts()

 VIN       124
 SIMPLE     43
 VI         24
Name:  model, dtype: int64

In [105]:
# This is not used
useful = [' algorithm', ' model', ' final_accuracy', ' performance']

In [106]:
# Create a column called 'perf' with the float values of the ' perfomance' column
compare_success = completed_frame.loc[:, (' performance')].apply(lambda x: float(x.split()[0]))
completed_frame['perf'] = compare_success

## Analysis: Useful functions

In [131]:
def get_rows_by_col(pdframe, colname, val):
    """Gets the rows where the colname meets some value"""
    return pdframe.loc[pdframe[colname]==val]

def get_nontrivial_columns(pdframe):
    """Gets the nontrivial columns from the pdframe"""
    nontrivial = []
    for column in pdframe.columns:
        if pdframe[column].unique().size > 1:
            nontrivial.append(column)
    return nontrivial

def get_trans_col(frame, grouped, func_name, colname):
    """Gets the row with the value given by grouped.transform(func_name)"""
    idx = grouped.transform(func_name) == frame[colname]
    return frame.loc[idx]

def print_basic_stats(pdframe, group_col, colname):
    """Prints basic statistics about grouped columns
    
    i.e. conditions underwhich max & min are achieved
    
    group_col –– the column we want to groupby (e.g. ' model')
    colname   –– the values we want to compare (e.g. 'perf')
    """
    pdgrouped = pdframe.groupby(group_col)[colname]
    max_row = get_trans_col(pdframe, pdgrouped, 'max', colname)
    min_row = get_trans_col(pdframe, pdgrouped, 'min', colname)
    
    print("\nMean performance of models is: ")
    print(pdgrouped.mean())
    print("\nStd performance of models is: ")
    print(pdgrouped.std())
    print("\nMax performance of models is: ")
    print(max_row)
    print("\nMin performance of models is: ")
    print(min_row)

## Analyzing: given_rewards

In [132]:
given_rewards = get_rows_by_col(completed_frame, ' algorithm', ' given_rewards')

In [141]:
# Get nontrivial columns of given_rewards, and compare by performance
cols = get_nontrivial_columns(given_rewards)
print_basic_stats(given_rewards[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.659607
 VIN       0.802909
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.458814
 VIN       0.131706
Name: perf, dtype: float64

Max performance of models is: 
      max_delay   hyperbolic_constant   imsize    model                seeds  \
6             5                   0.9        8      VIN      1-2-3-4-5-6-7-8   
287           7                   0.9        8   SIMPLE   1-2-3-5-8-13-21-34   

         final_accuracy          performance      perf  
6     90.12187501043081   0.9243739500762154  0.924374  
287   83.39140617847443   0.9978897509131921  0.997890  

Min performance of models is: 
      max_delay   hyperbolic_constant   imsize    model  \
143           5                   1.0        8   SIMPLE   
161           5                   1.0       12      VIN   

                     seeds      final_accuracy            performance  \
143     1-2-3-5-8-13-21-34   68.47812515497208   0.061897778

## Analyzing: no_rewards

In [142]:
no_rewards = get_rows_by_col(completed_frame, ' algorithm', ' no_rewards')

In [143]:
cols = get_nontrivial_columns(no_rewards)
print_basic_stats(no_rewards[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.579466
 VIN       0.745789
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.435274
 VIN       0.164640
Name: perf, dtype: float64

Max performance of models is: 
      max_delay   hyperbolic_constant   imsize    model                seeds  \
39            5                   0.9        8      VIN   1-2-3-5-8-13-21-34   
182           5                   1.0        8   SIMPLE      1-2-3-4-5-6-7-8   

         final_accuracy          performance      perf  
39    95.53593748249114   0.9487861779026734  0.948786  
182   82.89765620231628   0.9976912485982808  0.997691  

Min performance of models is: 
      max_delay   hyperbolic_constant   imsize    model  \
37            5                   0.9        8   SIMPLE   
339           7                   0.9       12      VIN   

                     seeds      final_accuracy           performance      perf  
37    3-5-8-13-21-34-55-89   81.10156248509884    0.

## Analyzing: boltzmann_planner

In [148]:
b_planner = get_rows_by_col(completed_frame, ' algorithm', ' boltzmann_planner')

In [149]:
cols = get_nontrivial_columns(b_planner)
print_basic_stats(b_planner[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.745769
 VIN       0.704370
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.366376
 VIN       0.190893
Name: perf, dtype: float64

Max performance of models is: 
      max_delay   hyperbolic_constant   imsize    model  \
73            5                   0.9        8   SIMPLE   
220           5                   1.0        8      VIN   

                     seeds      final_accuracy          performance      perf  
73    3-5-8-13-21-34-55-89   83.43671880662441   0.9978364895812957  0.997836  
220    2-3-5-8-13-21-34-55   95.20781250484288   0.9301018765560356  0.930102  

Min performance of models is: 
      max_delay   hyperbolic_constant   imsize    model                seeds  \
71            5                   0.9        8   SIMPLE   1-2-3-5-8-13-21-34   
219           5                   1.0        8      VIN   1-2-3-5-8-13-21-34   

         final_accuracy           performance      perf  
71    

## Analyzing: vi_inference

In [152]:
vi_alg = get_rows_by_col(completed_frame, ' algorithm', ' vi_inference')
cols = get_nontrivial_columns(vi_alg)
print_basic_stats(vi_alg[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.137255
 VI        0.892551
 VIN       0.024533
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.242810
 VI        0.071929
 VIN       0.017874
Name: perf, dtype: float64

Max performance of models is: 
      hyperbolic_constant   imsize    model                  seeds  \
112                   0.9        8      VIN    2-3-5-8-13-21-34-55   
115                   0.9        8       VI     1-2-3-5-8-13-21-34   
118                   0.9        8       VI        1-2-3-4-5-6-7-8   
253                   1.0        8   SIMPLE   3-5-8-13-21-34-55-89   
259                   1.0        8       VI     1-2-3-5-8-13-21-34   

              performance      perf  
112   0.06828133088948023  0.068281  
115    0.9918996228644773  0.991900  
118    0.9918996228644773  0.991900  
253    0.7335380404446011  0.733538  
259    0.9918996228644773  0.991900  

Min performance of models is: 
      hyperbolic_constant   imsize